<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/hw5__NosenkoArtem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

# Домашнее задание: Доступные LLM

В этом домашнем задании мы познакомимся с библиотекой transformers и разберемся, как можно open source пользоваться моделями.

In [ ]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers - 35 баллов

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и олучить следующий токен через жадную генерацию!

1. Для создания модели используйте метод `from_pretrained` у `AutoModelForCausalLM` и `AutoTokenizer`;
2. Чтобы токенизировать текст вызовите `tokenizer(text, return_tensors="pt")`, тогда вы получите словарь тензоров
3. Передайте его ключи и значения в качестве аргументов в `__call__` (forward) метод модели и получите logits размерности \[batch_size, seq_len, vocab_size\]
4. По logits предскажите следующий токен и детокенизируйте его с помощью `tokenizer.decode`

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [ ]:
def move_to_device(inputs, device):
    for k, v in inputs.items():
        inputs[k] = v.to(device)
    return inputs

In [ ]:

# ---- Ваш код здесь ----
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device) # Ваш код здесь
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device) # ваш код здесь


text = "This is a sample text"

# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!

inputs = tokenizer([text], return_tensors="pt")
inputs = move_to_device(inputs, device)

imputs_ids = inputs['input_ids']
mask = inputs['attention_mask']

outputs = model(imputs_ids)
logits = outputs['logits']
next_token_idx: int = torch.argmax(logits, dim=-1)[:, -1].item()


next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"


# ---- Конец кода ----


In [ ]:
next_token_idx

2393

## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации нескольких токенов сразу есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [ ]:
params_generation = {
    'max_new_tokens' : 10,
    'do_sample' : True,
    'temperature' : 0.9,
    'top_k' : 20,
    'repetition_penalty' : 1.2
}

In [ ]:
text = "This is still a sample text, but"
inputs = tokenizer(text, return_tensors="pt")
inputs = move_to_device(inputs, device)

results = []
for i in range(10):
    # ---- Ваш код здесь ----
    gens = model.generate(
        **inputs,
        **params_generation
    )
    genertaion: str = tokenizer.decode(gens.view(-1)) # сгенерированный текст
    results.append(genertaion)
    # ---- Конец кода ----


assert len(set(results)) > 1

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
results

['This is still a sample text, but if you want to see the full output then use',
 "This is still a sample text, but it's not all that difficult to figure out.",
 "This is still a sample text, but it should be quite clear that we've seen the",
 "This is still a sample text, but it's more than just another font and you can",
 "This is still a sample text, but it's not 100% complete as of yet;",
 'This is still a sample text, but the original looks rather crudely designed and quite difficult',
 "This is still a sample text, but I'll update soon. This was my first time",
 'This is still a sample text, but the original image was taken just last week.\n',
 'This is still a sample text, but it might be worth the time to look at more',
 "This is still a sample text, but the code isn't too obvious. But there are"]

## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side="left" в конструкторе токенизатора.

In [ ]:

# ---- Ваш код здесь ----
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          padding_side='left') # ваш код здесь
tokenizer.pad_token_id = tokenizer.eos_token_id
# ---- Конец кода ----


In [ ]:
texts = ["This is a sample text", "I'm really tired and this is just about"]


# ---- Ваш код здесь ----

# Внимание! В данном задании нужна жадная генерация!

# Соберите оба текста в один батч и положите результаты генерации в
# batched_generations
batched_generations: List[str] = []

for text in texts:
    inputs = tokenizer(text, return_tensors="pt")
    inputs = move_to_device(inputs, device)
    next_tokens = tokenizer.decode(model.generate(**inputs, max_new_tokens=10)[0])
    batched_generations.append(next_tokens)

# Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации
# в single_generations
single_generations: List[str] = []

...

# ---- Конец кода ----

assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b



## Скоринг, Perplixity - 10 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $\P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [ ]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, return_tensors="pt")

# ---- Ваш код здесь ----
inputs = tokenizer(text, ...)



with torch.no_grad():
    logits = model(...).logits
    ...
    # ваш код здесь!
    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    # 2. Превращаем logits в log_probs
    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    # для этого нам поможет torch.gather
    # 4. Считаем вероятности и perplexity!

# ---- Конец кода ----


print(text_P)
print(ppl)

# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl

## Вопросы - 5 баллов

**Ответьте на вопросы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.
2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.


In [ ]:
# ваш ответ тут
# ---- Ваш код здесь ----

# ---- Конец кода ----

# Chat-Models - 20 баллов

Теперь мы познакомимся с chat-моделями, т.е. с моделями, которые предоставляют возможность общаться с ними как с ассистентом. Эти модели не просто продолджают текст слева-направо, а дают ответ на заданный вопрос.

## Формат - 5 баллов

Все chat-модели принимают ответ в особом формате, который позволяет им различать, кому принадлежит фраза - пользователю (user) или модели (assistant).
Давайте попробуем подать модели вопрос без какого-либо форматирования.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct", torch_dtype=torch.half).to(device)

In [ ]:
text = "hello how are you"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)

for i in range(5):
    print(tokenizer.decode(model.generate(**move_to_device(inputs, device), max_new_tokens=20, use_cache=True, do_sample=True, pad_token_id=tokenizer.eos_token_id)[0]))
    print("====" * 3)


Видим, что текст зачастую разламывается:
1. Иногда модель продолжает текст как базовая LLM
2. Иногда пытается придумать роли спикерам и трасформироваться в формат диалога
3. Иногда просто выдает бессвязный текст.

Это происходит потому, что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели

In [ ]:
prefix = tokenizer.apply_chat_template(
    conversation=
    [
        {"role": "system", "content": "You are a helpful assistant, who always helps user"},
        {"role": "user", "content": "How to learn about LLMs?"},
        {"role": "assistant", "content": "You can always attend deepschool!"},
        {"role": "user", "content": "Thank you!"},
    ],
    tokenize=False)
print(prefix)

Как мы видим в тексте ходы и роли разделены тэгом `<|im_start|>`. В таком формате модель училась поддерживать диалог. Давайте отформатируем следующий диалог и подадим его в генерацию модели. Подробнее про apply_chat_template можно прочитать в [туториале](https://huggingface.co/docs/transformers/main/en/chat_templating#applychattemplate). Обратите внимание на опцию add_generation_prompt! Эта опция добавляет текст таким образом, чтобы в конце была очередь генерировать текст от лица модели. Давайте попробуем собрать диалог и сгенерировать моделью ответ.

In [ ]:
messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]


# ---- Ваш код здесь ----
prefix = tokenizer.apply_chat_template(...)
# ---- Конец кода ----
reference = """<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
hello<|im_end|>
<|im_start|>assistant
I'm good. How can I help you today<|im_end|>
<|im_start|>user
I love you<|im_end|>
<|im_start|>assistant
"""

assert prefix.strip() == reference.strip()


In [ ]:

# ---- Ваш код здесь ----
inputs = tokenizer(prefix, ...)
model.generate...
print(...)
# ---- Конец кода ----

# Benchmark - 15

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа (нумерация с нуля)

In [ ]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[0]

Наша задача здесь - выбрать моделью один из четырех ответов и получить точность больше 0.25.

Есть несколько вариантов, как это делать. **Эти варианты отличаются по сложности и являются взаимоисключающими. За подход с генерацией можно получить максимум 5 баллов, за подход со скорингом по 1 сэмплу можно получить только 10 баллов, а за подход со скорингом батчей можно получить все 15 баллов**

### Генерация ответа - 5 баллов

Можно генерировать ответ напрямую. Для этого нужно:
1. Составить историю диалога из qeustion и choices с помощью messages и apply_chat_template
1. Сгенерировать ответ
1. Соотнести сгенерированный ответ с одним из вариантов ответа

У этого подхода есть один важный недостаток - модель могут сгенерировать ответ, не являющийся одним из заданных вариантов ответа. Соотнесение такой генерации с ответом решается эвристиками и скорее всего приведет к множеству ошибок.


### Скоринг по сэмплам - 10 баллов

У нас есть вопрос и 4 варианта ответа в 4 набора messages и подсчитать вероятность $$P(choices_i | question)$$, то есть условную вероятность каждого ответа при заданном вопросе, т.е. сделать то же самое, что мы делали в задаче про вероятность текста и perplexity.

1. Берем текст и вариант ответа, собираем из них промпт c функции `sample_to_texts` (проще будет в этом задании обойтись без apply_chat_template)
2. Подаем это в модель через `model(**inputs)`, берем в выходах `logits`. С помощью logits и input_ids считаем вероятность токенов, которые мы подали модели.
3. Здесь опционально можно считать как вероятность всего текста, так и только вероятность $$P(choices_i | question)$$ Т.к. для всех 4х вариантов ответа у нас общий префикс, то его вероятность будет общей константной для всех ответов.
4. Выбираем ответ, которому была дана наибольшая вероятность нашей моделью.

В этом варианте легко получить номер ответа, который модель оценивает выше и не нужно применять эвристики.

### Скоринг батчами - 15 баллов

Этот вариант отличается от предыдущего только тем, что нужно скорить за раз не один сэмпл и 4 ответа к нему, а несколько сэмплов за раз, т.е. обрабатывать данные батчом.

Дополнительная сложность этого варианта заключается в том, что у нас возникают сэмплы различной длины, которые мы добиваем паддингами. **Паддинги не нужно учитывать при подсчете вероятностей, это служебные токены!**

Чтобы вероятность паддингов не влияла на итоговую вероятность текста, можно на этапе, где у вас подсчитаны все вероятности токенов (вместе с паддингами) взять `inputs["attention_mask"]` и "занулить" по нему вероятности паддингов (если вы считаете log_probs, если вы честно умножаете вероятности, тогда вероятности паддингов нужно поставить равными единице).

В качестве проверки точности можете проверить, что качество с батчом размера 1 не сильно отличается от батча размера 3 (не более, чем на 0.02)

In [ ]:
def sample_to_texts(sample):
    return [sample["question"] + " " + answer for answer in sample["choices"]]

def calc_acc(p, y):
    assert len(p) == len(y)
    return sum(pi == yi for pi, yi in zip(p, y)) / len(p)

In [ ]:
y_true = [sample["answer"] for sample in mmlu]

Считаем вероятности по одному question и choice

In [ ]:

# ---- Ваш код здесь ----
all_prompts = sum([sample_to_texts(mmlu[i]) for i in range(len(mmlu))], [])
assert len(all_prompts) == 400
tokenizer.pad_token = tokenizer.eos_token

...
# ---- Конец кода ----

## Ответьте на следующие вопросы (5 баллов в сумме):
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.
2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?


In [ ]:

# ---- Ваш код здесь ----
...
# ---- Конец кода ----